### Checking and upgrading the DKube SDK

In [ ]:
import pkg_resources, sys
try:
    dkube_sdk_version = float(pkg_resources.get_distribution("dkube").version)
except:
    dkube_sdk_version = 0 ## means the dkube sdk is not installed
if dkube_sdk_version < 3.7:
    !{sys.executable} -m pip install  git+https://github.com/oneconvergence/dkube.git@3.7 --user >/dev/null
%reset -f

### DKube Resources

In [ ]:
import time,json
from dkube.sdk import *
import os
from urllib.parse import urlparse

# Set up font definitions for output
class style:
   RED = '\033[91m\033[1m'
   BOLD = '\033[1m'
   END = '\033[0m'

## Assign the username & token for access to the serving cluster
DKUBE_USERNAME = os.environ.get("DKUBE_USER_LOGIN_NAME")
DKUBE_TOKEN = os.environ.get("DKUBE_USER_ACCESS_TOKEN") 
DKUBE_URL = os.environ.get("DKUBE_URL")

### User-Defined Variables

In [ ]:
## fill in the name for your Code, Dataset, & Model repos here

CODE_REPO_NAME = "insurance-pre"
DATASET_REPO_NAME = "insurance-pre"
MODEL_REPO_NAME = "insurance-pre"

### Create Dataset & Model

In [ ]:
# URL for the training dataset
DKUBE_DATASET_URL = "https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv"

# Create the DKube api variable
dkube_api = DkubeApi(URL=DKUBE_URL,token=DKUBE_TOKEN)

# Create Dataset
try:
    print(f"{style.BOLD}Creating Dataset \"{DATASET_REPO_NAME}\"")
    dataset = DkubeDataset(DKUBE_USERNAME, name=DATASET_REPO_NAME)
    dataset.update_dataset_source(source="pub_url")
    dataset.update_puburl_details(url=DKUBE_DATASET_URL, extract="False")
    dkube_api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")
    else:
        print(f"{style.BOLD}Training Dataset \"{DATASET_REPO_NAME}\" exists")

# Create Model
print(f"{style.BOLD}Creating Model \"{MODEL_REPO_NAME}\"")
try:
    model = DkubeModel(DKUBE_USERNAME, name=MODEL_REPO_NAME)
    model.update_model_source(source='dvs')
    dkube_api.create_model(model)
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
        else:
            print(f"{style.BOLD}Model \"{MODEL_REPO_NAME}\" exists")
    else:
        raise e

### Save the configuration info

In [ ]:
image_exp_config = {"DKUBE_USERNAME":DKUBE_USERNAME,
                    "DKUBE_TOKEN":DKUBE_TOKEN, 
                    "CODE_REPO_NAME":CODE_REPO_NAME,
                    "DATASET_REPO_NAME":DATASET_REPO_NAME,
                    "MODEL_REPO_NAME":MODEL_REPO_NAME,
                    "DKUBE_URL":DKUBE_URL}
%store image_exp_config